In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

#! pip install optuna

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

random_state = 101

In [ ]:
#https://forecastegy.com/posts/xgboost-hyperparameter-tuning-with-optuna/
#https://aetperf.github.io/2021/02/16/Optuna-+-XGBoost-on-a-tabular-dataset.html
#+ ChatGPT for debugging

# def objective_xgboost(trial):
#     params = {
#         "n_estimators": trial.suggest_int("max_depth", 100, 1000, step=50),
#         "verbosity": 3,
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
#         "max_depth": trial.suggest_int("max_depth", 1, 15),
#         "subsample": trial.suggest_float("subsample", 0.5, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
#         "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
#         "enable_categorical": True
#     }

#     model_xgbClass = xgb.XGBClassifier(**params, random_state=random_state, n_jobs=-1, tree_method="gpu_hist") #enable_categorical=True
    
#     skfold = StratifiedKFold(n_splits=3, random_state=random_state, shuffle=True)
#     auc_scores = []

#     for train_index, val_index in skfold.split(X_train, y_train):
#         X_train_fold, y_train_fold = X_train.iloc[train_index], y_train.iloc[train_index]
#         X_val_fold, y_val_fold = X_train.iloc[val_index], y_train.iloc[val_index]

#         model_xgbClass.fit(X_train_fold, y_train_fold, 
#                            eval_set = [(X_val_fold, y_val_fold)],
#                            early_stopping_rounds = 5,
#                            verbose = 3
#                            )
#         y_pred = model_xgbClass.predict(X_val_fold)#[:, 1]
#         auc_scores.append(roc_auc_score(y_val_fold, y_pred))

#     return np.mean(auc_scores)

In [ ]:
def objective_xgboost(trial):
    params = {
        "n_estimators": trial.suggest_int("max_depth", 100, 1000, step=50),
        "verbosity": 3,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "max_depth": trial.suggest_int("max_depth", 1, 15),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        "enable_categorical": True
    }

    model_xgbClass = xgb.XGBClassifier(**params, random_state=random_state, n_jobs=-1, tree_method="gpu_hist") #enable_categorical=True
    
    skfold = StratifiedKFold(n_splits=3, random_state=random_state, shuffle=True)
    
    scores = cross_val_score(model_xgbClass, X_train, y_train, cv=skfold, scoring="roc_auc")  

    return np.mean(scores)

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective_xgboost, n_trials=1) #Adjust trials

In [ ]:
best_params = study.best_params
final_model = xgb.XGBClassifier(**best_params, random_state=random_state, n_jobs=-1, enable_categorical=True)
final_model.fit(X_train, y_train)

In [ ]:
y_pred = final_model.predict(X_test)
test_auroc = roc_auc_score(y_test, y_pred)
print(test_auroc)

In [ ]:
final_model.save_model('xgboost_model.json')